In [ ]:
# Imports
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suprimir warnings TensorFlow

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pickle
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# TensorFlow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

# PyDMD
from pydmd import DMD

# Módulos propios
import sys
sys.path.append('../src')
from utils.metrics import evaluate_all

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('deep')

DATA_DIR = Path('../data/processed')
MODEL_DIR = Path('../data/models')
FIG_DIR = Path('../reports/figures')
RESULTS_DIR = DATA_DIR / 'experiments'
RESULTS_DIR.mkdir(exist_ok=True)

# Verificar GPU
print(f"✅ TensorFlow version: {tf.__version__}")
print(f"🔧 GPUs disponibles: {len(tf.config.list_physical_devices('GPU'))}")
if tf.config.list_physical_devices('GPU'):
    print(f"   GPU: {tf.config.list_physical_devices('GPU')[0].name}")

print(f"\n📁 Directorios:")
print(f"   Data: {DATA_DIR}")
print(f"   Models: {MODEL_DIR}")
print(f"   Experiments: {RESULTS_DIR}")

## **1. Cargar Datos Base**

Reutilizar el pipeline de datos del notebook 03.

In [ ]:
# Cargar datos procesados ERA5
nc_path = DATA_DIR / 'era5_precipitation_chile_full.nc'
ds = xr.open_dataset(nc_path)
ds_daily = ds.resample(time='1D').sum('time') * 1000  # m/día → mm/día

precip_data = ds_daily['tp'].values  # (366, 157, 41)
n_days, n_lat, n_lon = precip_data.shape

print(f"✅ Datos cargados:")
print(f"   Shape: {precip_data.shape}")
print(f"   Días: {n_days}, Grid: {n_lat}×{n_lon}")
print(f"   Media: {precip_data.mean():.2f} mm/día")
print(f"   Std: {precip_data.std():.2f} mm/día")

In [ ]:
# Normalizar datos
precip_flat = precip_data.reshape(n_days, -1)
scaler = StandardScaler()
precip_normalized = scaler.fit_transform(precip_flat)
X = precip_normalized.reshape(n_days, n_lat, n_lon, 1)

print(f"✅ Normalización completada:")
print(f"   X shape: {X.shape}")
print(f"   X mean: {X.mean():.6f} (cercano a 0)")
print(f"   X std: {X.std():.6f} (cercano a 1)")

In [ ]:
# Crear secuencias temporales
WINDOW_SIZE = 7

def create_sequences(data, window_size=7):
    X_seq, y_seq = [], []
    for i in range(len(data) - window_size):
        X_seq.append(data[i:i+window_size])
        y_seq.append(data[i+window_size])
    return np.array(X_seq), np.array(y_seq)

X_seq, y_seq = create_sequences(X, WINDOW_SIZE)

# Split train/val/test
n_train = int(0.7 * len(X_seq))
n_val = int(0.15 * len(X_seq))

X_train = X_seq[:n_train]
y_train = y_seq[:n_train]
X_val = X_seq[n_train:n_train+n_val]
y_val = y_seq[n_train:n_train+n_val]
X_test = X_seq[n_train+n_val:]
y_test = y_seq[n_train+n_val:]

print(f"✅ Secuencias creadas:")
print(f"   Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
print(f"   Window size: {WINDOW_SIZE} días")

In [ ]:
# Cargar pesos espaciales (kriging variance)
kriging_ds = xr.open_dataset(DATA_DIR / 'precipitation_kriging_june2020.nc')
kriging_variance = kriging_ds['kriging_variance'].values
spatial_weights = 1 / (kriging_variance + 1e-6)  # Inversa de varianza
spatial_weights = spatial_weights / spatial_weights.max()  # Normalizar [0, 1]
weights_tensor = tf.constant(spatial_weights, dtype=tf.float32)

print(f"✅ Pesos espaciales cargados:")
print(f"   Shape: {spatial_weights.shape}")
print(f"   Range: [{spatial_weights.min():.3f}, {spatial_weights.max():.3f}]")

## **2. Función de Experimento Automatizada**

Wrapper para entrenar y evaluar modelos con diferentes configuraciones.

In [ ]:
def build_autoencoder(latent_dim=64, dilations=[1,2,4,8], l2_reg=0.0001):
    """Construir autoencoder con parámetros configurables."""
    input_shape = (n_lat, n_lon, 1)
    
    # ENCODER
    encoder_input = keras.Input(shape=input_shape, name='spatial_input')
    x = encoder_input
    
    for i, dilation in enumerate(dilations):
        x = layers.Conv2D(
            filters=32,
            kernel_size=3,
            dilation_rate=dilation,
            padding='same',
            activation='relu',
            kernel_regularizer=keras.regularizers.l2(l2_reg),
            name=f'conv{i+1}_d{dilation}'
        )(x)
    
    x = layers.Flatten()(x)
    encoded = layers.Dense(
        latent_dim,
        activation='relu',
        kernel_regularizer=keras.regularizers.l2(l2_reg),
        name='latent'
    )(x)
    
    encoder = keras.Model(encoder_input, encoded, name='encoder')
    
    # DECODER
    decoder_input = keras.Input(shape=(latent_dim,), name='latent_input')
    x = layers.Dense(
        n_lat * n_lon,
        activation='relu',
        kernel_regularizer=keras.regularizers.l2(l2_reg)
    )(decoder_input)
    x = layers.Reshape((n_lat, n_lon, 1))(x)
    
    for i, dilation in enumerate(reversed(dilations)):
        x = layers.Conv2D(
            filters=32,
            kernel_size=3,
            dilation_rate=dilation,
            padding='same',
            activation='relu',
            kernel_regularizer=keras.regularizers.l2(l2_reg)
        )(x)
    
    decoded = layers.Conv2D(
        1,
        kernel_size=3,
        padding='same',
        activation='linear',
        name='output'
    )(x)
    
    decoder = keras.Model(decoder_input, decoded, name='decoder')
    
    # AUTOENCODER COMPLETO
    autoencoder_input = keras.Input(shape=input_shape)
    encoded = encoder(autoencoder_input)
    decoded = decoder(encoded)
    autoencoder = keras.Model(autoencoder_input, decoded, name='autoencoder')
    
    return encoder, decoder, autoencoder

print("✅ Función build_autoencoder definida")

In [ ]:
def weighted_mse_loss(y_true, y_pred):
    """Loss MSE ponderada por pesos espaciales."""
    squared_diff = tf.square(y_true - y_pred)
    weighted_squared_diff = squared_diff * weights_tensor
    return tf.reduce_mean(weighted_squared_diff)

print("✅ Loss function definida")

In [ ]:
def run_experiment(config, experiment_id):
    """Ejecutar un experimento completo con configuración dada."""
    print(f"\n{'='*80}")
    print(f"🧪 EXPERIMENTO {experiment_id}: {config['name']}")
    print(f"{'='*80}")
    
    start_time = time.time()
    
    # 1. Build model
    encoder, decoder, autoencoder = build_autoencoder(
        latent_dim=config['latent_dim'],
        dilations=config['dilations'],
        l2_reg=config.get('l2_reg', 0.0001)
    )
    
    autoencoder.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss=weighted_mse_loss
    )
    
    print(f"✅ Modelo construido: latent_dim={config['latent_dim']}, dilations={config['dilations']}")
    
    # 2. Train autoencoder
    X_train_single = X_train[:, -1, :, :, :]  # Último frame
    X_val_single = X_val[:, -1, :, :, :]
    
    early_stop = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )
    
    history = autoencoder.fit(
        X_train_single, X_train_single,
        validation_data=(X_val_single, X_val_single),
        epochs=config.get('epochs', 100),
        batch_size=config.get('batch_size', 32),
        callbacks=[early_stop],
        verbose=0
    )
    
    train_time = time.time() - start_time
    print(f"✅ Entrenamiento completado en {train_time:.1f}s")
    print(f"   Final train loss: {history.history['loss'][-1]:.6f}")
    print(f"   Final val loss: {history.history['val_loss'][-1]:.6f}")
    print(f"   Épocas: {len(history.history['loss'])}")
    
    # 3. Encode to latent space
    latent_train = encoder.predict(X_train[:, -1], verbose=0)
    latent_test = encoder.predict(X_test[:, -1], verbose=0)
    
    # 4. Train DMD
    X_snapshots = latent_train[:-1].T
    Y_snapshots = latent_train[1:].T
    
    svd_rank = config.get('svd_rank', 0.99)
    dmd = DMD(svd_rank=svd_rank)
    dmd.fit(X_snapshots)
    
    n_modes = dmd.modes.shape[1]
    eigs_magnitude = np.abs(dmd.eigs)
    n_stable = np.sum(eigs_magnitude < 1.0)
    
    print(f"✅ DMD entrenado: {n_modes} modos, {n_stable} estables ({100*n_stable/n_modes:.1f}%)")
    
    # 5. Forecasting 1-step
    z0 = latent_test[0]
    Lambda = np.diag(dmd.eigs)
    Phi = dmd.modes
    Phi_inv = np.linalg.pinv(Phi)
    A_dmd = Phi @ Lambda @ Phi_inv
    
    latent_forecasts = []
    z_current = z0
    for _ in range(len(latent_test)):
        z_next = A_dmd @ z_current
        latent_forecasts.append(z_next.real)
        z_current = z_next
    
    latent_forecasts = np.array(latent_forecasts)
    
    # 6. Decode to spatial
    spatial_forecasts = decoder.predict(latent_forecasts, verbose=0)
    
    # 7. Desnormalizar
    spatial_forecasts_flat = spatial_forecasts.reshape(-1, n_lat * n_lon)
    spatial_forecasts_real = scaler.inverse_transform(spatial_forecasts_flat)
    spatial_forecasts_real = spatial_forecasts_real.reshape(-1, n_lat, n_lon, 1)
    
    y_test_flat = y_test.reshape(-1, n_lat * n_lon)
    y_test_real = scaler.inverse_transform(y_test_flat)
    y_test_real = y_test_real.reshape(-1, n_lat, n_lon, 1)
    
    # 8. Métricas
    mae = np.mean(np.abs(spatial_forecasts_real - y_test_real))
    rmse = np.sqrt(np.mean((spatial_forecasts_real - y_test_real) ** 2))
    
    print(f"✅ Forecasting completado:")
    print(f"   MAE:  {mae:.3f} mm/día")
    print(f"   RMSE: {rmse:.3f} mm/día")
    
    # 9. Guardar resultados
    results = {
        'experiment_id': experiment_id,
        'config': config,
        'train_time': train_time,
        'train_loss': history.history['loss'][-1],
        'val_loss': history.history['val_loss'][-1],
        'epochs': len(history.history['loss']),
        'n_modes': n_modes,
        'n_stable_modes': n_stable,
        'mae': mae,
        'rmse': rmse,
        'timestamp': datetime.now().isoformat()
    }
    
    print(f"\n✅ Experimento completado en {time.time() - start_time:.1f}s")
    
    return results

print("✅ Función run_experiment definida")

## **3. Definir Grid de Experimentos**

Combinaciones de hiperparámetros a explorar.

In [ ]:
# Grid de experimentos
experiments = []

# Experimento 1: Baseline (configuración actual)
experiments.append({
    'name': 'Baseline',
    'latent_dim': 64,
    'dilations': [1, 2, 4, 8],
    'svd_rank': 0.99,
    'epochs': 100,
    'batch_size': 32
})

# Experimentos 2-5: Variar latent_dim
for latent_dim in [32, 128, 256]:
    experiments.append({
        'name': f'LatentDim_{latent_dim}',
        'latent_dim': latent_dim,
        'dilations': [1, 2, 4, 8],
        'svd_rank': 0.99,
        'epochs': 100,
        'batch_size': 32
    })

# Experimentos 6-9: Variar SVD rank
for svd_rank in [0.90, 0.95, 1.0]:
    experiments.append({
        'name': f'SVDRank_{svd_rank:.2f}',
        'latent_dim': 64,
        'dilations': [1, 2, 4, 8],
        'svd_rank': svd_rank,
        'epochs': 100,
        'batch_size': 32
    })

# Experimentos 10-11: Variar dilations
experiments.append({
    'name': 'Dilations_1_3_9_27',
    'latent_dim': 64,
    'dilations': [1, 3, 9, 27],
    'svd_rank': 0.99,
    'epochs': 100,
    'batch_size': 32
})

experiments.append({
    'name': 'Dilations_1_2_4',
    'latent_dim': 64,
    'dilations': [1, 2, 4],
    'svd_rank': 0.99,
    'epochs': 100,
    'batch_size': 32
})

# Experimentos 12-14: Variar epochs
for epochs in [50, 150]:
    experiments.append({
        'name': f'Epochs_{epochs}',
        'latent_dim': 64,
        'dilations': [1, 2, 4, 8],
        'svd_rank': 0.99,
        'epochs': epochs,
        'batch_size': 32
    })

# Experimentos combinados (mejores de cada categoría)
experiments.append({
    'name': 'Combined_LargeDim_HighRank',
    'latent_dim': 128,
    'dilations': [1, 2, 4, 8],
    'svd_rank': 1.0,
    'epochs': 100,
    'batch_size': 32
})

experiments.append({
    'name': 'Combined_SmallDim_LowRank',
    'latent_dim': 32,
    'dilations': [1, 2, 4, 8],
    'svd_rank': 0.90,
    'epochs': 100,
    'batch_size': 32
})

print(f"✅ Grid de experimentos definido: {len(experiments)} configuraciones")
print(f"\n📋 Resumen:")
for i, exp in enumerate(experiments, 1):
    print(f"   {i:2d}. {exp['name']}")

## **4. Ejecutar Experimentos**

⚠️ **NOTA**: Esto tomará varias horas. Se recomienda ejecutar en sesiones separadas.

In [ ]:
# Ejecutar todos los experimentos
all_results = []

for i, config in enumerate(experiments, 1):
    try:
        results = run_experiment(config, experiment_id=i)
        all_results.append(results)
        
        # Guardar resultados incrementalmente
        with open(RESULTS_DIR / 'experiments_results.pkl', 'wb') as f:
            pickle.dump(all_results, f)
        
        print(f"💾 Progreso guardado: {i}/{len(experiments)} experimentos")
        
    except Exception as e:
        print(f"❌ Error en experimento {i}: {e}")
        continue

print(f"\n🎉 TODOS LOS EXPERIMENTOS COMPLETADOS: {len(all_results)}/{len(experiments)} exitosos")

## **5. Análisis de Resultados**

Visualizar y comparar todos los experimentos.

In [ ]:
# Cargar resultados (si se ejecutó en sesión anterior)
# with open(RESULTS_DIR / 'experiments_results.pkl', 'rb') as f:
#     all_results = pickle.load(f)

# Convertir a DataFrame
df_results = pd.DataFrame([
    {
        'experiment_id': r['experiment_id'],
        'name': r['config']['name'],
        'latent_dim': r['config']['latent_dim'],
        'svd_rank': r['config']['svd_rank'],
        'dilations': str(r['config']['dilations']),
        'epochs': r['config']['epochs'],
        'train_time': r['train_time'],
        'train_loss': r['train_loss'],
        'val_loss': r['val_loss'],
        'n_modes': r['n_modes'],
        'mae': r['mae'],
        'rmse': r['rmse']
    }
    for r in all_results
])

# Ordenar por MAE
df_results = df_results.sort_values('mae')

print("="*100)
print("📊 RESULTADOS DE TODOS LOS EXPERIMENTOS")
print("="*100)
print(df_results[['experiment_id', 'name', 'latent_dim', 'svd_rank', 'mae', 'rmse', 'train_time']].to_string())
print("="*100)

# Guardar CSV
df_results.to_csv(RESULTS_DIR / 'experiments_summary.csv', index=False)
print(f"\n💾 Resultados guardados: {RESULTS_DIR / 'experiments_summary.csv'}")

In [ ]:
# Top 5 mejores configuraciones
print("\n🏆 TOP 5 MEJORES CONFIGURACIONES (por MAE):")
print("="*80)
top5 = df_results.head(5)
for i, row in enumerate(top5.itertuples(), 1):
    emoji = '🥇' if i == 1 else '🥈' if i == 2 else '🥉' if i == 3 else ''
    print(f"{emoji} #{row.experiment_id}: {row.name}")
    print(f"   MAE: {row.mae:.3f} mm/día, RMSE: {row.rmse:.3f} mm/día")
    print(f"   Latent: {row.latent_dim}, SVD rank: {row.svd_rank}, Modos: {row.n_modes}")
    print(f"   Train time: {row.train_time:.1f}s\n")

In [ ]:
# Visualizaciones
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# 1. MAE vs Latent Dim
latent_dims = df_results['latent_dim'].unique()
mae_by_latent = df_results.groupby('latent_dim')['mae'].mean()
axes[0].bar(latent_dims, mae_by_latent, alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Latent Dimension', fontsize=11)
axes[0].set_ylabel('MAE (mm/día)', fontsize=11)
axes[0].set_title('MAE vs Latent Dimension', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# 2. MAE vs SVD Rank
svd_ranks = sorted(df_results['svd_rank'].unique())
mae_by_svd = df_results.groupby('svd_rank')['mae'].mean()
axes[1].plot(svd_ranks, mae_by_svd, marker='o', linewidth=2, markersize=8)
axes[1].set_xlabel('SVD Rank', fontsize=11)
axes[1].set_ylabel('MAE (mm/día)', fontsize=11)
axes[1].set_title('MAE vs SVD Rank', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# 3. MAE vs RMSE scatter
axes[2].scatter(df_results['mae'], df_results['rmse'], alpha=0.6, s=100)
axes[2].set_xlabel('MAE (mm/día)', fontsize=11)
axes[2].set_ylabel('RMSE (mm/día)', fontsize=11)
axes[2].set_title('MAE vs RMSE', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)

# 4. Train time vs MAE
axes[3].scatter(df_results['train_time'], df_results['mae'], alpha=0.6, s=100)
axes[3].set_xlabel('Train Time (s)', fontsize=11)
axes[3].set_ylabel('MAE (mm/día)', fontsize=11)
axes[3].set_title('Efficiency: Train Time vs MAE', fontsize=12, fontweight='bold')
axes[3].grid(True, alpha=0.3)

# 5. N Modes vs MAE
axes[4].scatter(df_results['n_modes'], df_results['mae'], alpha=0.6, s=100)
axes[4].set_xlabel('Number of DMD Modes', fontsize=11)
axes[4].set_ylabel('MAE (mm/día)', fontsize=11)
axes[4].set_title('DMD Modes vs MAE', fontsize=12, fontweight='bold')
axes[4].grid(True, alpha=0.3)

# 6. Top 10 experiments bar chart
top10 = df_results.head(10)
axes[5].barh(range(len(top10)), top10['mae'], alpha=0.7, edgecolor='black')
axes[5].set_yticks(range(len(top10)))
axes[5].set_yticklabels([f"{row.experiment_id}: {row.name}" for row in top10.itertuples()], fontsize=9)
axes[5].set_xlabel('MAE (mm/día)', fontsize=11)
axes[5].set_title('Top 10 Experiments', fontsize=12, fontweight='bold')
axes[5].invert_yaxis()
axes[5].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(FIG_DIR / 'hyperparameter_analysis.png', dpi=150, bbox_inches='tight')
print(f"💾 Guardado: {FIG_DIR / 'hyperparameter_analysis.png'}")
plt.show()

---

## **Conclusiones**

Este notebook permite:

1. **Exploración sistemática** de hiperparámetros
2. **Comparación objetiva** de configuraciones
3. **Identificación de trade-offs** (performance vs tiempo)
4. **Selección de configuración óptima** para producción

**Próximos pasos:**
- Re-entrenar modelo óptimo con más épocas
- Validar configuración óptima en años 2019-2021
- Registrar mejor modelo en MLflow

**Nota**: Este notebook puede tardar 2-4 horas en ejecutarse completamente.